In [ ]:
%load_ext autoreload
%autoreload 2
import pudl
import logging
import sys
from pathlib import Path
import pandas as pd
import calendar
import requests
from tqdm import tqdm

pd.options.display.max_columns = None

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()

if you don't have tqdm you can comment it out or install it `mamba install tqdm -c conda-forge`

# Extraction testing

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pudl_settings = pudl.workspace.setup.get_defaults()
from pudl.settings import Eia860Settings, Eia923Settings, EiaSettings
from pudl.metadata.classes import DataSource

eia860_data_source = DataSource.from_id("eia860")
eia860_years = eia860_data_source.working_partitions["years"]
#eia860_years = [2020]
eia860_settings = Eia860Settings(years=eia860_years)

# Uncomment to use all available years:
eia923_data_source = DataSource.from_id("eia923")
eia923_years = eia923_data_source.working_partitions["years"]
#eia923_years = [2020]
eia923_settings = Eia923Settings(years=eia923_years)

eia_settings = EiaSettings(eia860=eia860_settings, eia923=eia923_settings)
ds_kwargs = {"local_cache_path": pudl_settings["data_dir"]}
ds = pudl.workspace.datastore.Datastore(**ds_kwargs)

## 860m

In [ ]:
eia860m_extractor = pudl.extract.eia860m.Extractor(ds)
eia860m_raw_dfs = eia860m_extractor.extract()

## 860

In [ ]:
eia860_extractor = pudl.extract.eia860.Extractor(ds)
eia860_raw_dfs = eia860_extractor.extract()

# Setting up 860m stuff
## Semi-automated way of populating skipfooter, skiprows, and column_maps

In [ ]:
l = list(
    Path(pudl_settings["data_dir"])
    .joinpath("eia860m/10.5281-zenodo.6321197")
    .glob("*.xlsx")
)


df_f = pd.DataFrame()
df_r = pd.DataFrame()
cols = {0: [], 1: [], 2: []}
f = l[0]
for f in tqdm(l):
    xl = pd.ExcelFile(f)
    name = f.stem.partition("-")[-1]
    for sht in range(3):
        df = xl.parse(sheet_name=sht)
        df_r.loc[sht, name] = df.index[df.iloc[:, 0] == "Entity ID"][0] + 1
        df_f.loc[sht, name] = df.iloc[500:, 0].isna().sum() + 1
        df.columns = list(df.loc[df.index[df.iloc[:, 0] == "Entity ID"][0], :])
        df = pudl.helpers.simplify_columns(df)
        cols[sht].append(df.columns.to_series(name=name))


pd.concat(cols[2], axis=1).sort_index(axis=1).to_clipboard()
q = pd.DataFrame(sorted(x.name for x in l))
q.index = [x.partition("-")[2].replace(".xlsx", "") for x in q[0]]

## for working_partitions in `metadata.sources`

In [ ]:
print([f"20{y}-{m:02}" for y in range(15, 23) for m in range(1, 13)][6:][:-6])

## Grab missing months

In [ ]:
df = pd.read_clipboard()

# Helpers for column maps

In [ ]:
df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602020/6_2_EnviroEquip_Y2020.xlsx", header=1, sheet_name=0)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602019/6_2_EnviroEquip_Y2019.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602018/6_2_EnviroEquip_Y2018.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602017/6_2_EnviroEquip_Y2017.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602016/6_2_EnviroEquip_Y2016.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602015/6_2_EnviroEquip_Y2015.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602014/6_2_EnviroEquip_Y2014.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602013/6_2_EnviroEquip_Y2013.xlsx", header=1, sheet_name=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602012/EnviroEquipY2012.xlsx", header=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602011/EnviroEquip.xlsx", header=1)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602010/EnviroEquipY2010.xls", header=0)
# df = pd.read_excel("/Users/aengel/Downloads/EIA/eia8602009/EnviroEquipY09.xls", header=0)
pudl.helpers.simplify_columns(df).head().T.to_clipboard()

In [ ]:
df.head()